In [1]:
import dask
import sys
sys.path.append('/HistomicsTK/server/cli_common/')
import utils as cli_utils
import histomicstk.preprocessing.color_normalization as htk_cnorm
import large_image
import histomicstk.utils as htk_utils
from skimage.measure import regionprops
from skimage.segmentation import slic
import numpy as np
import h5py
from ctk_cli import CLIArgumentParser
import dask.distributed as dd
import logging
logging.basicConfig(level=logging.CRITICAL)
import histomicstk.segmentation as htk_seg
from skimage.measure import regionprops
from skimage.segmentation import slic

INFO:girder:Created LRU Cache for 'tilesource' with 104 maximum size
INFO:root:Notice: Could not import MapnikTileSource


In [2]:
## slides we want to run on are in 
slidePath = img_path= "/data/train/TCGA-AA-3544-01Z-00-DX1.96850cbf-2305-4b65-8f06-db801af51cc3.svs"
ts = large_image.getTileSource(slidePath)
# compute colorspace statistics (mean, variance) for whole slide
wsi_mean, wsi_stddev = htk_cnorm.reinhard_stats(slidePath, 0.1, 10)

INFO:girder:Cannot use memcached for caching.
INFO:girder:Using python for large_image caching


Using python for large_image caching


In [3]:
print(wsi_mean,wsi_stddev)

(array([ 8.73684822, -0.23802296,  0.01820436]), array([0.41583865, 0.12890332, 0.01723846]))


In [4]:
def compute_superpixel_data(img_path, tile_position, wsi_mean, wsi_stddev):
    
    # get slide tile source
    ts = large_image.getTileSource(img_path)

    # get requested tile information
    tile_info = ts.getSingleTile(
        tile_position=tile_position,
        resample=True,
        format=large_image.tilesource.TILE_FORMAT_NUMPY)
    
    
    im_tile = tile_info['tile'][:, :, :3]

    # get global x and y positions
    left = tile_info['gx']
    top = tile_info['gy']

    # get scale
    scale = tile_info['gwidth'] / tile_info['width']

    reference_mu_lab = [8.63234435, -0.11501964, 0.03868433]
    reference_std_lab = [0.57506023, 0.10403329, 0.01364062]
    
    # perform color normalization
    im_nmzd = htk_cnorm.reinhard(im_tile,
                                 reference_mu_lab, reference_std_lab,
                                 wsi_mean, wsi_stddev)
    patchSize = 32
    # compute the number of super-pixels
    im_width, im_height = im_nmzd.shape[:2]
    n_superpixels = (im_width/patchSize) * (im_height/patchSize)

    #
    # Generate labels using a superpixel algorithm (SLIC)
    # In SLIC, compactness controls image space proximity.
    # Higher compactness will make the shape of superpixels more square.
    #
    
    compactness = 50
    im_label = slic(im_nmzd, n_segments=n_superpixels,compactness=compactness) + 1

    region_props = regionprops(im_label)

    # set superpixel data list
    # set superpixel data list
    s_data = []
    x_cent = []
    y_cent = []
    x_brs = []
    y_brs = []

    for i in range(len(region_props)):
        # get x, y centroids for superpixel
        cen_x, cen_y = region_props[i].centroid

        # get bounds of superpixel region
        min_row, max_row, min_col, max_col = \
            get_patch_bounds(cen_x, cen_y, patchSize, im_width, im_height)

        # grab superpixel label mask
        lmask = (im_label[:, :] == region_props[i].label).astype(np.bool)

        # embed with center pixel in middle of padded window
        emask = np.zeros((lmask.shape[0] + 2, lmask.shape[1] + 2), dtype=np.bool)
        emask[1:-1, 1:-1] = lmask

        # find boundaries
        bx, by = htk_seg.label.trace_object_boundaries(emask)

        with np.errstate(invalid='ignore'):
            # remove redundant points
            mby, mbx = htk_utils.merge_colinear(by[0].astype(float), bx[0].astype(float))

        scaled_x = (mbx - 1) * scale
        scaled_y = (mby - 1) * scale

        # get superpixel boundary at highest-res
        x_brs.append(scaled_x + top)
        y_brs.append(scaled_y + left)

        
        rgb_data = im_nmzd[min_row:max_row, min_col:max_col]

        s_data.append(rgb_data)
        
        # get superpixel centers at highest-res
        x_cent.append(
            round((cen_x * scale + top), 1))
        y_cent.append(
            round((cen_y * scale + left), 1))

    return s_data, x_cent, y_cent, x_brs, y_brs

def get_patch_bounds(cx, cy, patch_size, m, n):

    half_patch_size = patch_size/2.0

    min_row = int(round(cx) - half_patch_size)
    max_row = int(round(cx) + half_patch_size)
    min_col = int(round(cy) - half_patch_size)
    max_col = int(round(cy) + half_patch_size)

    if min_row < 0:
        max_row = max_row - min_row
        min_row = 0

    if max_row > m-1:
        min_row = min_row - (max_row - (m-1))
        max_row = m-1

    if min_col < 0:
        max_col = max_col - min_col
        min_col = 0

    if max_col > n-1:
        min_col = min_col - (max_col - (n-1))
        max_col = n-1

    return min_row, max_row, min_col, max_col

In [5]:
# compute tissue/foreground mask at low-res for whole slide images
im_fgnd_mask_lres, fgnd_seg_scale = cli_utils.segment_wsi_foreground_at_low_res(ts)

In [6]:
# compute foreground fraction of tiles in parallel using Dask
analysis_tile_size = 2048
analysis_mag = 10
it_kwargs = {
    'tile_size': {'width': analysis_tile_size},
    'scale': {'magnification': analysis_mag},
}

inputSlidePath = 0

tile_fgnd_frac_list = htk_utils.compute_tile_foreground_fraction(
    slidePath, im_fgnd_mask_lres, fgnd_seg_scale,
    **it_kwargs
)

print('\n>> Detecting superpixel data ...\n')


>> Detecting superpixel data ...



In [7]:
tile_result_list = []
min_fgnd_frac = 0.001


for tile in ts.tileIterator(**it_kwargs):
    tile_position = tile['tile_position']['position']
    if tile_fgnd_frac_list[tile_position] <= min_fgnd_frac:
        continue

    # detect superpixel data
    cur_result = compute_superpixel_data(slidePath,tile_position,wsi_mean, wsi_stddev)

    # append result to list
    tile_result_list.append(cur_result)

#tile_result_list = tile_result_list.compute()

# initiate output data list

/build/miniconda/lib/python2.7/site-packages/histomicstk-0.2.0-py2.7-linux-x86_64.egg/histomicstk/utils/merge_colinear.py:36: RuntimeWarning: divide by zero encountered in divide
  slope = dY / dX


In [9]:
slideBaseName = "TCGA-AA-3544-01Z-00-DX1.96850cbf-2305-4b65-8f06-db801af51cc3"

In [12]:
import os
print slidePath
slideBaseName = os.path.basename(slidePath).split(".")[0]
print slideBaseName

/data/train/TCGA-AA-3544-01Z-00-DX1.96850cbf-2305-4b65-8f06-db801af51cc3.svs
TCGA-AA-3544-01Z-00-DX1


In [13]:
superpixel_data = []
x_centroids = []
y_centroids = []
x_boundaries = []
y_boundaries = []

for s_data, x_cent, y_cent, x_brs, y_brs in tile_result_list:
        for s_d in s_data:
            superpixel_data.append(s_d)

        for x_c in x_cent:
            x_centroids.append(x_c)

        for y_c in y_cent:
            y_centroids.append(y_c)

        for x_b in x_brs:
            x_boundaries.append(x_b)

        for y_b in y_brs:
            y_boundaries.append(y_b)

superpixel_data = np.asarray(superpixel_data, dtype=np.float32)

n_superpixels = len(superpixel_data)
x_centroids = np.asarray(x_centroids).reshape((n_superpixels, 1))
y_centroids = np.asarray(y_centroids).reshape((n_superpixels, 1))

#
# Last, we can store the data
#
print('>> Writing superpixel data information')

output = h5py.File('%s.spixelfeatures'% slideBaseName, 'w')
output.create_dataset('features', data=superpixel_data)
output.create_dataset('x_centroid', data=x_centroids)
output.create_dataset('y_centroid', data=y_centroids)
output.close()

>> Writing superpixel data information


In [14]:
superpixel_data

array([[[[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        ...,

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
       

In [15]:
print('>> Writing text boundary file')

boundary_file = open('%s.BoundariesFile' % slideBaseName, 'w')
boundary_file.write("x_centroid  , y_centroid  , Four boundary coordinates\n")
for i in range(n_superpixels):
    boundary_file.write("%.1f\t" % y_centroids[i, 0])
    boundary_file.write("%.1f\t" % x_centroids[i, 0])
    for j in range(len(x_boundaries[i])):
        boundary_file.write("%d,%d " % (y_boundaries[i][j], x_boundaries[i][j]))
    boundary_file.write("\n")
boundary_file.close()

>> Writing text boundary file


In [18]:
boundary_file

<closed file 'TCGA-AA-3544-01Z-00-DX1.BoundariesFile', mode 'w' at 0x7f8622314ae0>